In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

In [3]:
# ResNet 모델 불러오기 (예: ResNet-18)
model = models.resnet18(pretrained=True)
# 모델의 fully connected layer를 새로운 레이어로 대체
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 4)  # 4개의 클래스에 맞게 출력 뉴런 개수를 설정


C:\Users\20161\anaconda3\envs\torch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\20161\anaconda3\envs\torch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
# 데이터셋 및 데이터 로더 설정
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])
train_dataset = datasets.ImageFolder(root='E:/PycharmProjects/peslab/GDSC SC/recycle_img/Training', transform=transform)
test_dataset = datasets.ImageFolder(root='E:/PycharmProjects/peslab/GDSC SC/recycle_img/Validation', transform=transform)

In [5]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [6]:
# GPU 사용 가능하다면 GPU로 모델을 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [7]:
# 손실 함수와 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [8]:
num_epochs = 5 # 학습 에폭 수 설정
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    s = 0
    n = 1
    for inputs, labels in train_loader:
        # inputs = inputs.unsqueeze(0).expand(3, -1, -1, -1, -1)
        inputs, labels = inputs.to(device), labels.to(device)
        print(inputs.shape)
        # inputs = inputs.permute(0, 2, 1, 3, 4)
        optimizer.zero_grad()
        print(labels)
        # print(labels.shape)
        outputs = model(inputs)
        loss = F.cross_entropy(outputs, labels)
        # loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        s += len(train_loader)
        print("Completed : ", n, "The rest :", len(train_loader)-n)
        n += 1
print("Training complete!")

torch.Size([64, 3, 64, 64])
tensor([3, 3, 2, 3, 2, 3, 2, 2, 2, 2, 3, 3, 2, 2, 2, 3, 3, 2, 3, 2, 2, 2, 2, 0,
        2, 2, 2, 3, 3, 2, 2, 2, 2, 2, 3, 3, 0, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 0,
        3, 2, 0, 3, 3, 3, 2, 0, 2, 3, 2, 3, 2, 3, 2, 3], device='cuda:0')
Completed :  1 The rest : 729
torch.Size([64, 3, 64, 64])
tensor([3, 2, 2, 3, 1, 2, 3, 3, 1, 0, 2, 3, 3, 3, 2, 2, 3, 3, 2, 3, 0, 3, 2, 2,
        2, 3, 3, 0, 3, 3, 2, 3, 2, 3, 2, 2, 2, 3, 1, 2, 3, 1, 2, 3, 3, 2, 3, 3,
        2, 1, 2, 3, 2, 1, 2, 2, 3, 2, 2, 2, 3, 2, 3, 2], device='cuda:0')
Completed :  2 The rest : 728
torch.Size([64, 3, 64, 64])
tensor([3, 3, 2, 3, 0, 3, 2, 2, 3, 0, 3, 2, 3, 3, 2, 3, 2, 2, 3, 2, 3, 3, 3, 2,
        3, 3, 2, 2, 2, 2, 2, 3, 3, 3, 2, 2, 1, 3, 3, 3, 2, 3, 2, 1, 2, 3, 2, 3,
        3, 2, 2, 3, 2, 3, 2, 2, 3, 2, 2, 3, 2, 3, 2, 3], device='cuda:0')
Completed :  3 The rest : 727
torch.Size([64, 3, 64, 64])
tensor([3, 3, 2, 3, 2, 2, 3, 2, 3, 2, 3, 3, 3, 3, 2, 3, 3, 0, 0, 3, 0, 2, 3, 2,
        3, 2, 3,

In [9]:
# 모델 평가
def evaluate_model(loader, model):
    model.eval()  # 모델을 평가 모드로 전환
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            print(labels)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total * 100
    return accuracy

# 테스트 데이터로 모델 평가
test_accuracy = evaluate_model(test_loader, model)
print(f"Test Accuracy: {test_accuracy:.2f}%")

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [12]:
torch.save(model.state_dict(), 'E:/PycharmProjects/peslab/GDSC SC/resnet18_recycle_epoch5.pth')

In [ ]:
torch.save(model.state_dict(), 'E:/PycharmProjects/peslab/GDSC SC/resnet18_recycle_epoch5.pth')

In [13]:
model_scripted = torch.jit.script(model) # TorchScript 형식으로 내보내기
model_scripted.save('E:/PycharmProjects/peslab/GDSC SC/resnet18_recycle_epoch5.pt') # 저장하기

In [22]:
import torch
import torch.onnx

# 모델을 평가 모드로 설정
model.eval()

# 더미 입력을 준비합니다. 이는 모델의 입력 크기에 맞춰야 합니다.
dummy_input = torch.randn(1,3,5,4)

In [23]:
# 모델을 ONNX로 변환하고 "model.onnx" 파일로 저장
torch.onnx.export(model, dummy_input, "model.onnx", verbose=True, input_names=['input'], output_names=['output'])

RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor

In [20]:
save_path ='E:/PycharmProjects/peslab/GDSC SC/resnet18_recycle_epoch7_weights.pth'
torch.save(model.state_dict(), save_path)
print("Model has been saved.")

Model has been saved.


In [1]:
pip install requests Pillow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
import requests
from PIL import Image
from io import BytesIO

# 이미지 URL
image_url = "https://firebasestorage.googleapis.com/v0/b/thethethe-ba342.appspot.com/o/images%2F1707898016167.jpg?alt=media&token=050e5891-5218-43de-b837-89d3c2d05f86"

In [12]:
# 이미지 다운로드
response = requests.get(image_url)

In [13]:
response

<Response [200]>

In [14]:
image = Image.open(BytesIO(response.content))

In [15]:

# 이미지 정보 출력
print(f"Image format: {image.format}")
print(f"Image size: {image.size}")
print(f"Image mode: {image.mode}")

# 이미지 보기 (주피터 노트북이나 콘솔에서 실행하는 경우)
image.show()

# 추가적인 이미지 처리 작업...
# 예: 이미지 크기 조정, 회전, 모델 입력 데이터로 변환 등


Image format: JPEG
Image size: (1080, 2400)
Image mode: RGB


In [16]:
image.show()